In [10]:
import numpy as np
from qutip import*
sqrt = np.sqrt
pi   = np.pi
import Chalmers_Simulator_mesolve as CS
class Add_gate:
    def __init__(gate, name, Tar_Con, angle):
        gate.name    = name
        gate.Tar_Con = Tar_Con
        gate.angle   = angle

In [35]:
Nqubits= 2
Nlevels= 3
Paulis_gt = 2.00e-8
CZ_gt = 2.71e-7
Diss = [35.98e-6, 36.24e-6]
Deph = [38.74e-6, 39.34e-6]
Alpha = -2*pi*200e6
ZZ_list = [[0,1]]
ZZ_strength = [2*pi*100e3]
Hamiltonian, c_ops = CS.create_system_Hamiltonian(Nqubits, Nlevels, Paulis_gt, CZ_gt, 0, Alpha, Diss, \
                                                  Deph, Texc = [], ZZ_list=[], ZZ_strength=[])

Sz = Qobj(np.array(np.array([[1, 0, 0], [0, -1, 0], [0, 0, 1]])))
Sx = Qobj(np.array(np.array([[0, 1, 0], [1, 0, 0], [0, 0, 1]])))
Un = qeye(3)

# Calculating the cost function
H1=  -1.0537076071291114*  tensor(Un,  Un)
H2=  0.3939836794385142*   tensor(Un,  Sz)
H3= -0.3939836794385142*   tensor(Sz,  Un)
H4= -0.011236585233181995* tensor(Sz,  Sz)
H5=  0.181288808211496*    tensor(Sx,  Sx)
cost_function= H1 + H2 + H3 + H4 + H5

Theta_range= np.linspace(0,2*pi,50)
state=[]
cost=[]
Ini = tensor(basis(3,0), basis(3,0))

In [36]:
CF00 = [0.968, 0.020, 0.011, 0.000]
CF01 = [0.059, 0.930, 0.010, 0.011]
CF10 = [0.059, 0.001, 0.921, 0.019]
CF11 = [0.040, 0.057, 0.056, 0.884]
CM = Qobj([CF00, CF01, CF10, CF11])

In [ ]:
coeff = [-1.0537076071291114, 0.3939836794385142, -0.3939836794385142, \
         -0.011236585233181995, 0.181288808211496]
cost = []
for i, theta in enumerate(Theta_range):
    
    gate=[]
    gate.append(Add_gate(name=["PX" , "HD"], Tar_Con=[0,1],   angle=[pi, 0]           ))
    gate.append(Add_gate(name=["PX" , "PZ"], Tar_Con=[0,1],   angle=[pi/2, theta]     ))
    gate.append(Add_gate(name=["HD"       ], Tar_Con=[1],     angle=[0]               ))
    gate.append(Add_gate(name=["CZ"       ], Tar_Con=[[0,1]], angle=[0]               ))
    gate.append(Add_gate(name=["PX"       ], Tar_Con=[0],     angle=[-pi/2]           ))
    fstate = CS.Execute(Hamiltonian, c_ops, gate, Ini)    
    
    # Measurment part begins
    gate=[]
    gate.append(Add_gate(name=["U" , "U"],  Tar_Con=[0,1],   angle=[0, 0]           ))
    gate.append(Add_gate(name=["U" , "PZ"], Tar_Con=[0,1],   angle=[0, 0]           ))
    gate.append(Add_gate(name=["PZ" , "U"], Tar_Con=[0,1],   angle=[0, 0]           ))
    gate.append(Add_gate(name=["PZ" , "PZ"],Tar_Con=[0,1],   angle=[0, 0]           ))
    gate.append(Add_gate(name=["PY" , "PY"],Tar_Con=[0,1],   angle=[-pi/2, -pi/2]     ))
    cost.append(np.sum(np.real(CS.Measurement(Hamiltonian, fstate, gate, CM, coeff))))
    if np.mod(i+1, 10) == 0:
        print(i+1, 'of', len(Theta_range), 'completed')

10 of 50 completed
20 of 50 completed


In [ ]:
import matplotlib.pyplot as plt
plt.plot(Theta_range, cost)


import pickle
with open('Cost_func_expt','rb') as read_file:
    expt = pickle.load(read_file)
theta_expt = np.linspace(0,2*pi,len(expt))

plt.plot(theta_expt, expt,'*')
plt.grid('on')

In [19]:
len(cost)

50

In [30]:
0.3939836794385142*(0.006 - 0.945 - 0.040 + 0.009)

-0.38216416905535877

In [24]:
tensor( sigmaz(), qeye(2))

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0. -1.]]

In [10]:
cost[0]

[(0.9786515756786764-9.207967612780648e-16j),
 (-0.3659200579605762+3.442880108000849e-16j),
 (0.3659200579605762-3.442880108000849e-16j),
 (0.01043619858991288-9.819243232692286e-18j),
 (0.055758672128711474+6.526739871638421e-17j)]

In [56]:
tensor(sigmaz(), sigmaz())

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.  0.  0.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  1.]]

In [46]:
Qobj(fstate.diag())

Quantum object: dims = [[9], [1]], shape = (9, 1), type = ket
Qobj data =
[[5.37022379e-03]
 [9.05095593e-01]
 [2.73367058e-04]
 [3.80004735e-02]
 [8.95625929e-03]
 [3.36728161e-02]
 [1.28601542e-03]
 [6.13547956e-03]
 [1.20977191e-03]]

In [22]:
def final_pop(probs, gate):
    
    ops = []
    for i in gate:
        if i=='PX' or i=='PY' or i=='PZ':
            ops.append(sigmaz())
            
        elif i == 'U':
            ops.append(qeye(2))
            
            
    ops= tensor(ops).diag()

    sum = 0
    for i, j in enumerate(ops):
        sum = sum + j*probs[i]

    return sum 

In [26]:
tensor(sigmax(), qeye(2)).diag()

array([0., 0., 0., 0.])

In [14]:
import itertools as it
def _3to2levels(dm):
    '''
    This function returns the density matrix in the computational subspace.
    
    Arguments-
    dm          :      Density matrix in the total Hilbert space
    
    Returns-
    Prob_array  :      Array of probabilities in the computational space
    
    '''


    levels= list(map(",".join, it.product(*[map(str, range(Nlevels))])))
    states = ["".join(seq) for seq in it.product(levels, repeat=Nqubits)]
    Prob_array = []
    counter = 0
    for i in states:
        if '2' not in i:
            Prob_array.append(dm[counter, counter])
        counter = counter + 1
    
    Prob_array = np.array(Prob_array)
    norm = 1/np.sum(Prob_array)
    return Prob_array